In [1]:
import pickle as pk
import requests

import bs4 as bs
from bs4 import XMLParsedAsHTMLWarning
import warnings

warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

In [2]:
url = 'https://github.com/fadeeva/EDGAR_research/raw/refs/heads/master/data/reports.pickle'
response = requests.get(url)
reports = pk.loads(response.content)

len(reports.keys())

496

In [3]:
reports['AAPL']

['https://www.sec.gov/Archives/edgar/data/320193/000032019325000057/aapl-20250329.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019325000008/aapl-20241228.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019324000081/aapl-20240629.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019324000069/aapl-20240330.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019324000006/aapl-20231230.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000077/aapl-20230701.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000064/aapl-20230401.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019323000006/aapl-20221231.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019322000070/aapl-20220625.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019322000059/aapl-20220326.htm',
 'https://www.sec.gov/Archives/edgar/data/320193/000032019322000007/aapl-20211225.htm',
 'https://www.sec.gov/Archives/e

In [14]:
headers={'User-Agent':'Dummy Company jane.dow@dummy.com','Accept-Encoding':'gzip, deflate','Host':'www.sec.gov'}

response = requests.get('https://www.sec.gov/Archives/edgar/data/320193/000032019325000057/aapl-20250329.htm',
                        headers=headers, allow_redirects=True)

corpus = bs.BeautifulSoup(response.text, features='lxml')

In [15]:
ticker, date = corpus.find('title').text.split('-')
ticker, date

('aapl', '20250329')

In [16]:
corpus.find(id='f-5').text # Commission File Numbe

'001-36743'

In [17]:
corpus.find(id='f-8').text # I.R.S. Employer Identification No

'94-2404110'

In [18]:
# Securities registered pursuant to Section 12(b) of the Act:

# Title of each class
corpus.find(id='f-15').text
corpus.find(id='f-18').text
corpus.find(id='f-21').text
corpus.find(id='f-24').text
corpus.find(id='f-27').text
corpus.find(id='f-30').text
corpus.find(id='f-33').text
corpus.find(id='f-36').text
corpus.find(id='f-39').text


'Common Stock, $0.00001 par value per share'